In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [33]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.83it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [34]:
hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_max_date.name = 'hist_purchase_date_max'
hist_min_date.name = 'hist_purchase_date_min'
hist_max_date = hist_max_date.to_frame()
hist_min_date = hist_min_date.to_frame()

# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: 
                                                                           str(int(str(x)[:4])+1) + '-' + '01' 
                                                                              if str(x)[:7][-2:]=='12' 
                                                                           else str(x)[:4] + '-0' + str(int(str(x)[:7][-1:])+1)
                                                                           if str(x)[:7][-2:]!='09' 
                                                                           else str(x)[:4] + '-10')

hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])

df_hist.set_index(key, inplace=True)
df_hist = df_hist.join(hist_max_date.join(hist_min_date))

new_max_date = df_new.groupby(key)['purchase_date'].max()
new_min_date = df_new.groupby(key)['purchase_date'].min()
new_max_date.name = 'new_purchase_date_max'
new_min_date.name = 'new_purchase_date_min'
new_max_date = new_max_date.to_frame()
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: 
                                                                           str(int(str(x)[:4])+1) + '-' + '01' 
                                                                              if str(x)[:7][-2:]=='12' 
                                                                           else str(x)[:4] + '-0' + str(int(str(x)[:7][-1:])+1)
                                                                           if str(x)[:7][-2:]!='09' 
                                                                           else str(x)[:4] + '-10')
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_new.set_index(key, inplace=True)
df_new = df_new.join(new_max_date).join(new_min_date)

In [43]:
# train_test = train_test.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
train_test['hist_purchase_month_max'] = pd.to_datetime(train_test['hist_purchase_month_max'])
train_test['hist_purchase_month_min'] = pd.to_datetime(train_test['hist_purchase_month_min'])
train_test['new_purchase_month_max'] = pd.to_datetime(train_test['new_purchase_month_max'])
train_test['new_purchase_month_min'] = pd.to_datetime(train_test['new_purchase_month_min'])
train_test.head()

,feature_1,feature_2,feature_3,first_active_month,target,hist_purchase_date_max,hist_purchase_month_max,hist_purchase_date_min,hist_purchase_month_min,new_purchase_date_max,new_purchase_month_max,new_purchase_date_min,new_purchase_month_min
card_id,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,2018-02-25 09:31:15,2018-03-01,2017-06-27 14:18:08,2017-06-01,2018-04-29 11:23:05,2018-05-01,2018-03-05 14:04:36,2018-03-01
C_ID_3d0044924f,4,1,0,2017-01,0.392913,2018-01-31 22:31:09,2018-02-01,2017-01-06 16:29:42,2017-01-01,2018-03-30 06:48:26,2018-04-01,2018-02-01 17:07:54,2018-02-01
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,2018-02-27 19:08:25,2018-03-01,2017-01-11 08:21:22,2017-01-01,2018-04-28 17:43:11,2018-05-01,2018-04-28 17:43:11,2018-04-01
C_ID_186d6a6901,4,3,0,2017-09,0.142495,2018-02-28 11:44:40,2018-03-01,2017-09-26 16:22:21,2017-09-01,2018-04-18 11:00:11,2018-05-01,2018-03-07 11:55:06,2018-03-01
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,2018-02-28 20:40:41,2018-03-01,2017-11-12 00:00:00,2017-11-01,2018-04-28 18:50:25,2018-05-01,2018-03-02 11:55:43,2018-03-01


In [54]:
# 静的なFeature
train_test['personal_term_new_min_hist_max'] = (train_test['new_purchase_month_min'] - train_test['hist_purchase_month_max']).dt.days
train_test['personal_term_new_min_hist_min'] = (train_test['new_purchase_month_min'] - train_test['hist_purchase_month_min']).dt.days
train_test['personal_days_new_max_date_from_hist_max_month'] = (train_test['new_purchase_date_max'] - train_test['hist_purchase_month_max']).dt.days
train_test['personal_days_new_min_date_from_hist_max_month'] = (train_test['new_purchase_date_min'] - train_test['hist_purchase_month_max']).dt.days
        
prefix = '203_pst'
ignore_features = ['first_active_month', 'card_id', target]

for col in train_test.columns:
    if col in ignore_features: continue
    if not(col.count('personal')):continue
    feature = train_test[col].astype('float32').values
    
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)

In [48]:
df_hist['hist_purchase_month_max'] = pd.to_datetime(df_hist['hist_purchase_month_max'])
df_hist['hist_purchase_month_min'] = pd.to_datetime(df_hist['hist_purchase_month_min'])
df_hist['new_purchase_month_max'] = pd.to_datetime(df_hist['new_purchase_month_max'])
df_hist['new_purchase_month_min'] = pd.to_datetime(df_hist['new_purchase_month_min'])

df_new['new_purchase_month_max'] = pd.to_datetime(df_new['new_purchase_month_max'])
df_new['new_purchase_month_min'] = pd.to_datetime(df_new['new_purchase_month_min'])
df_new['hist_purchase_month_max'] = pd.to_datetime(df_new['hist_purchase_month_max'])
df_new['hist_purchase_month_min'] = pd.to_datetime(df_new['hist_purchase_month_min'])

# display(df_hist.head())
# display(df_new.head())

In [49]:
df_new_lag1 = df_new[df_new['month_lag']==1]
df_new_lag2 = df_new[df_new['month_lag']==2]

df_hist_lag0 = df_hist[df_hist['month_lag']==0]
df_hist_lag1 = df_hist[df_hist['month_lag']==-1]
df_hist_lag2 = df_hist[df_hist['month_lag']==-2]
df_hist_lag02 = df_hist[df_hist['month_lag']>=-2]
df_hist_lag05 = df_hist[df_hist['month_lag']>=-5]

In [58]:
prefix = '203_pst'
new_df_list = [df_new, df_new_lag1, df_new_lag2]
new_fname_list = ['new', 'new_lag1', 'new_lag2']
hist_df_list = [df_hist, df_hist_lag0, df_hist_lag1, df_hist_lag2, df_hist_lag02, df_hist_lag05] 
hist_fname_list = ['hist', 'hist_lag0', 'hist_lag1', 'hist_lag2', 'hist_lag02', 'hist_lag05']

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
#========================================================================
# 集計が必要なFeature
#========================================================================
for df, fname in zip(new_df_list, new_fname_list):
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    # new_first_buyと同じだな
    # df['personal_new_elapsed_month'] = (df['new_purchase_month_min'] - df['first_active_month']).dt.days
    df['diff_date_from_new_min_month'] = (df['new_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_new_max_month'] = (df['new_purchase_month_max'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_min_month'] = (df['hist_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_max_month'] = (df['hist_purchase_month_max'] - df['purchase_date']).dt.days
    
    aggs = {}
    aggs['diff_date_from_new_min_month'] = ['mean', 'max', 'min', 'std', 'skew']
    aggs['diff_date_from_new_max_month'] = ['mean', 'max', 'min', 'std', 'skew']
    aggs['diff_date_from_hist_min_month'] = ['mean', 'max', 'min', 'std', 'skew']
    aggs['diff_date_from_hist_max_month'] = ['mean', 'max', 'min', 'std', 'skew']
    
    df.reset_index(inplace=True)
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    train_test = train_test.join(df_agg)
    
    #========================================================================
    # Save Feature
    ignore_features = ['first_active_month', 'card_id', target]
    for col in train_test.columns:
        if col in ignore_features: continue
        if not(col.count('diff')):continue
        feature = train_test[col].astype('float32').values
        
    #     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
    #========================================================================
    
    
for df, fname in zip(hist_df_list, hist_fname_list):
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    # new_first_buyと同じだな
    # df['personal_new_elapsed_month'] = (df['new_purchase_month_min'] - df['first_active_month']).dt.days
    df['diff_date_from_new_min_month'] = (df['new_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_new_max_month'] = (df['new_purchase_month_max'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_min_month'] = (df['hist_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_max_month'] = (df['hist_purchase_month_max'] - df['purchase_date']).dt.days
    
    aggs = {}
    aggs['diff_date_from_new_min_month'] = ['mean', 'max', 'min', 'std', 'skew']
    aggs['diff_date_from_new_max_month'] = ['mean', 'max', 'min', 'std', 'skew']
    aggs['diff_date_from_hist_min_month'] = ['mean', 'max', 'min', 'std', 'skew']
    aggs['diff_date_from_hist_max_month'] = ['mean', 'max', 'min', 'std', 'skew']
    
    df.reset_index(inplace=True)
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    train_test = train_test.join(df_agg)
    
    #========================================================================
    # Save Feature
    ignore_features = ['first_active_month', 'card_id', target]
    for col in train_test.columns:
        if col in ignore_features: continue
        if not(col.count('diff')):continue
        feature = train_test[col].astype('float32').values
        
    #     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
    #========================================================================

100%|██████████| 3/3 [00:00<00:00, 173.89it/s]


In [47]:
train_test['new_purchase_month_max'].value_counts()

2018-05-01    167857
2018-04-01     55349
2018-03-01     16554
2018-02-01     11655
2018-01-01      8537
2017-02-01      6396
2017-01-01      5411
2017-10-01      4657
2017-09-01      4014
2017-08-01      3602
2017-07-01      3069
2017-06-01      2441
2017-05-01       434
2017-04-01        25
Name: new_purchase_month_max, dtype: int64